In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths

from subprocess import check_output

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical

C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_images = glob("C:/Users/jdu12/Desktop/humpback/train/*jpg")
test_images = glob("C:/Users/jdu12/Desktop/humpback/test/*jpg")
df = pd.read_csv("C:/Users/jdu12/Desktop/humpback/train.csv")

#df["Image"] = df["Image"].map( lambda x : "C:/Users/jdu12/Desktop/humpback/input/train/"+x)
#ImageToLabelDict = dict( zip( df["Image"], df["Id"]))

In [11]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(224, 224))#.convert('L')
    img = img_to_array(img)
    #img = preprocess_input(img)
    return img

In [12]:
# accquire images' paths
trainPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/train/'))
print(len(trainPaths))
train = np.array([get_image(img_path) for img_path in trainPaths])
print(train.shape)

9850
(9850, 224, 224, 3)


In [13]:
#a = load_img(trainPaths[0], target_size=(299, 299))
#a.show()

In [14]:
def to_one_hot(labels):
    one_hot = []
    seen_id = set()
    id2class = dict()
    counter = 0
    for id in labels:
        if id not in seen_id:
            seen_id.add(id)
            one_hot.append(counter)
            id2class[id] = counter
            counter += 1
        else:
            one_hot.append(id2class[id])
    one_hot = to_categorical(one_hot, num_classes = 4251)
    return one_hot

In [7]:
labels = df['Id']
labels = to_one_hot(labels)
#print(labels[0:10])

In [8]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation


In [16]:
# define data generator
#use of an image generator for preprocessing and data augmentation
#train = train.reshape( (-1,299,299,1))
input_shape = train[0].shape
x_train = train.astype("float32")
y_train = labels

# define data generator
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

In [15]:
# acquire the model
model = keras.applications.densenet.DenseNet121(include_top=True, weights=None, classes=4251)


In [ ]:
batch_size = 32
epochs = 200

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = 'C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/'
routine_dir = model_dir + "routine-{epoch:02d}-{acc:.2f}.hdf5"
routine_save = ModelCheckpoint(routine_dir, monitor='acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=10)
model.fit_generator(image_gen.flow(x_train, y_train, batch_size=batch_size),
          steps_per_epoch=  x_train.shape[0]//batch_size,
          epochs=epochs, verbose=1, callback=[routine_save])
          #class_weight=class_weight_dic)



Epoch 1/200
307/307 [==============================] - 120s 390ms/step - loss: 5.6699 - acc: 0.1038
Epoch 2/200
307/307 [==============================] - 105s 342ms/step - loss: 4.7876 - acc: 0.1285
Epoch 3/200
307/307 [==============================] - 105s 342ms/step - loss: 4.1200 - acc: 0.1677
Epoch 4/200
307/307 [==============================] - 105s 342ms/step - loss: 3.5708 - acc: 0.2236
Epoch 5/200
307/307 [==============================] - 105s 342ms/step - loss: 3.0354 - acc: 0.3207
Epoch 6/200
307/307 [==============================] - 105s 342ms/step - loss: 2.6025 - acc: 0.3915
Epoch 7/200
307/307 [==============================] - 105s 342ms/step - loss: 2.2354 - acc: 0.4653
Epoch 8/200
307/307 [==============================] - 105s 343ms/step - loss: 1.9268 - acc: 0.5198
Epoch 9/200
307/307 [==============================] - 105s 343ms/step - loss: 1.7021 - acc: 0.5707
Epoch 10/200
307/307 [==============================] - 105s 342ms/step - loss: 1.4531 - acc: 0.6308

In [ ]:
# save model
model.save('C:/Users/jdu12/Desktop/humpback/saved_model/DenseNet/' + 'first_DenseNet.hdf5')
#model.save(model_dir + 'first_DenseNet.hdf5')


In [ ]:
# testing and generating submission file
